In [41]:
from scipy.io import loadmat
data = loadmat('trainingData.mat')
data = data['dataRecord']


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.backends.quantized.engine = 'qnnpack'
# 假设你的数据是numpy数组形式，需要转换成torch Tensor
import numpy as np

# 示例数据，你应该用你的实际数据替换这里的np.random.rand()
inputs = torch.tensor(data[:, :6], dtype=torch.float32)
outputs = torch.tensor(data[:, 6:], dtype=torch.float32)

# 创建数据集和数据加载器
dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义神经网络模型
class SimpleFNN(nn.Module):
    def __init__(self):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(6, 6)  # 输入层到隐藏层，50个神经元
        self.fc2 = nn.Linear(6, 3)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = SimpleFNN()
model.train()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练网络
epochs = 200
for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.00716536957770586
Epoch 2, Loss: 0.0004481541982386261
Epoch 3, Loss: 1.7308573660557158e-05
Epoch 4, Loss: 1.2335419341980014e-05
Epoch 5, Loss: 1.2424581655068323e-05
Epoch 6, Loss: 1.5629648260073736e-05
Epoch 7, Loss: 1.4640002518717665e-05
Epoch 8, Loss: 5.9366566347307526e-06
Epoch 9, Loss: 9.684975339041557e-06
Epoch 10, Loss: 9.33116280066315e-06
Epoch 11, Loss: 1.2220090866321698e-05
Epoch 12, Loss: 1.1390774488972966e-05
Epoch 13, Loss: 1.3102335287840106e-05
Epoch 14, Loss: 8.006356438272633e-06
Epoch 15, Loss: 1.6263727957266383e-05
Epoch 16, Loss: 1.0973289136018138e-05
Epoch 17, Loss: 7.630269465153106e-06
Epoch 18, Loss: 8.973458534455858e-06
Epoch 19, Loss: 1.0174457202083431e-05
Epoch 20, Loss: 1.6382404282921925e-05
Epoch 21, Loss: 9.571595001034439e-06
Epoch 22, Loss: 1.3766283700533677e-05
Epoch 23, Loss: 9.533281627227552e-06
Epoch 24, Loss: 6.182252491271356e-06
Epoch 25, Loss: 1.19787300718599e-05
Epoch 26, Loss: 9.97828919935273e-06
Epoch 27, Lo

In [43]:
# infer and save
test_input = torch.tensor([[0.11412254, -0.09698317, -0.13648682, -0.08168529,  0.09584415,
        -0.08081015]], dtype=torch.float32)  # 示例输入数据
with torch.no_grad():
    prediction = model(test_input)
    print(f'Prediction: {prediction.numpy()}')
    print("-0.01443261, -0.01519005, -0.02687549")
# save the model
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, '../model/model.pth')

Prediction: [[-0.01487498 -0.01549172 -0.0268057 ]]
-0.01443261, -0.01519005, -0.02687549


In [44]:
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
print(model.qconfig)

# 准备量化
torch.quantization.prepare(model, inplace=True)

# 假设输入进行校准
model(inputs)

# 转换为量化模型
quantized_model = torch.quantization.convert(model, inplace=True)

# 保存为TorchScript
scripted_model = torch.jit.script(quantized_model)
scripted_model.save("quantized_model.pth")

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})


/Users/bing/anaconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [45]:
outputs = quantized_model(inputs)
print(outputs)

NotImplementedError: Could not run 'quantized::linear' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear' is only available for these backends: [MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

MPS: registered at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFa

In [27]:
data = loadmat('testingData.mat')
test_data = data['dataRecord']

# 测试数据集，应该与训练数据集具有相同的结构
test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_outputs = torch.tensor(test_data[:, 6:], dtype=torch.float32)

# 创建测试 DataLoader
test_dataset = TensorDataset(test_inputs, test_outputs)
test_loader = DataLoader(test_dataset, batch_size=10)

# 将模型设置为评估模式
model.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')


NotImplementedError: Could not run 'quantized::linear' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear' is only available for these backends: [MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

MPS: registered at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFa